## Objective

Let's fine-tune our LassoCV model and interpret it.

In [2]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type

In [3]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [4]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

../preprocessing/helper_functions.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[pd.np.abs(group - group.median()) > stds * group.std()] = pd.np.nan
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.where(-key, value, inplace=True)


(508653, 40)

I'm removing ~1.5% of my rows.

## Choosing columns

In [5]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'TYPE', 'SubmittedPhoto', 'Property_Type', 'Source', 'neighborhood_from_zip']
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [6]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]

## Dummify

In [7]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'TYPE', u'Property_Type', u'Source', u'neighborhood_from_zip',
       u'school', u'housing'],
      dtype='object')

In [8]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

Zoning is baseline 0 6
other is baseline 1 6
Twitter is baseline 2 6
West Roxbury is baseline 3 6
8_6th_grade is baseline 4 6
rent is baseline 5 6


In [9]:
df_dummified.shape

(508653, 253)

## Running model

In [10]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [12]:
pipe = make_pipeline(StandardScaler(), LassoCV())

In [13]:
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [14]:
grid_searches = []
best_models = []

for alpha in [1e-5, 3e-4, 1e-4, 3e-3, 1e-3, 3e-2, 1e-2, 3e-1, 1e-1, 3]:
    params = {'lassocv__alphas': [[alpha]]}
    model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=75)
    model.fit(X_train, y_train)
    
    grid_searches += [model]
    best_models += [model.best_estimator_.steps[-1][-1]]

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Pickling array (shape=(252,), dtype=object).
Pickling array (shape=(406922,), dtype=object).
Pickling array (shape=(2, 406922), dtype=bool).
Memmaping (shape=(245, 406922), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_4220_140617543544464/4220-140617543681296-d64f0471daabfc947c8dc25c8895f76a.pkl
Memmaping (shape=(5, 406922), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_4220_140617543544464/4220-140617543681296-389b7c8adcba4caca8208567368f53bd.pkl
Pickling array (shape=(2,), dtype=object).
Pickling array (shape=(245,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(245,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(406922,), dtype=object).
Memmaping (shape=(406922,), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_4220_140617543544464/4220-140617543681296-a4af92d7b27e92d6205638ded29d1c26.pkl
Pickling array (shape=(406922,), dty

Process PoolWorker-3:
Process PoolWorker-4:
Traceback (most recent call last):
Process PoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self._target(*self._args, **self._kwargs)
    self.run()
    self.run()
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    task = get()
    self._target(

KeyboardInterrupt: 

In [145]:
def make_alphas_table(models, grid_searches):
    X = [model.alphas_[0] for model in models]    
    y = [g.cv_results_['mean_test_score'][0] for g in grid_searches]
        
    fixed_order_indices = [0, 2, 1, 4, 3, 6, 5, 8, 7, 9]
    X = np.array(X)[fixed_order_indices]
    y = np.array(y)[fixed_order_indices]
        
    return X, y

In [147]:
X_alphas, y_alphas = make_alphas_table(best_models, saved_models)

In [161]:
ax = plt.gca()
ax.plot(X_alphas, y_alphas)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim())
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('$R^2$')
plt.title('As regularization increases, performance decreases (along with # of features)');

The best hyperparameter value is α=0.01, which surprisingly is what we got during our first try!

## Interpreting our α=0.01 model

Using the results from our first try.

In [179]:
# model_best = grid_searches[-4]

In [180]:
model_best.best_params_

{'lassocv__alphas': [0.01]}

115 of the 252 coefficients go to 0.

In [182]:
pd.Series(model_best.best_estimator_.steps[-1][-1].coef_).describe()

count    252.000000
mean      -0.004569
std        0.049632
min       -0.363193
25%        0.000000
50%       -0.000000
75%        0.000000
max        0.145756
dtype: float64

In [27]:
sorted(list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ != 0]))

['Source_Citizens Connect App',
 'Source_Self Service',
 'TYPE_Abandoned Bicycle',
 'TYPE_Abandoned Building',
 'TYPE_Abandoned Vehicles',
 'TYPE_Animal Found',
 'TYPE_Animal Generic Request',
 'TYPE_Animal Lost',
 'TYPE_Bed Bugs',
 'TYPE_Bicycle Issues',
 'TYPE_Breathe Easy',
 'TYPE_Building Inspection Request',
 'TYPE_Call Log',
 'TYPE_Carbon Monoxide',
 'TYPE_Catchbasin',
 'TYPE_Checkin',
 'TYPE_Chronic Dampness/Mold',
 'TYPE_Construction Debris',
 'TYPE_Contractors Complaint',
 'TYPE_Cross Metering - Sub-Metering',
 'TYPE_Egress',
 'TYPE_Electrical',
 'TYPE_Empty Litter Basket',
 'TYPE_Equipment Repair',
 'TYPE_Exceeding Terms of Permit',
 'TYPE_General Comments For An Employee',
 'TYPE_General Comments For a Program or Policy',
 'TYPE_General Lighting Request',
 'TYPE_Graffiti Removal',
 'TYPE_Heat - Excessive  Insufficient',
 'TYPE_Highway Maintenance',
 'TYPE_Housing Discrimination Intake Form',
 'TYPE_Illegal Auto Body Shop',
 'TYPE_Illegal Dumping',
 'TYPE_Illegal Occupancy',


We will want to run a model with just the above features to find out which ones are statistically significant, but we get a sense here that these factors are likely to be signficant:

- when source is from the mobile app or desktop website
- neighborhoods of East Boston and the North End
- the number of issues in the workers' queue at the time

## Conclusion

We didn't get a better $R^2$, which makes sense, since we weren't in an overfit situation anyways when we tried this regularization parameter.

We did find subset our features and got somewhat of an indication which ones are more likely to be significantly correlated to completion time than others. We also avoided crazy predictions that would have affected our $R^2$, at least for this particular random seed.

## Appendix
### Graph of coefficients going to 0

In [127]:
def make_coefficients_table(models, n_coefs=10):
    X = [model.alphas_[0] for model in models]    
    y = [model.coef_[:n_coefs] for model in models]
        
    fixed_order_indices = [0, 2, 1, 4, 3, 6, 5, 8, 7, 9]
    X = np.array(X)[fixed_order_indices]
    y = np.array(y)[fixed_order_indices]
        
    return X, y

In [139]:
num_coefs = 20
X, y = make_coefficients_table(best_models, num_coefs)

ax = plt.gca()
ax.plot(X, y)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim())
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('standardized coefficients')
plt.title('{} Lasso coefficients as a function of the regularization'.format(num_coefs));

In [158]:
num_coefs = 500
X, y = make_coefficients_table(best_models, num_coefs)

ax = plt.gca()
ax.plot(X, y)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim())
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('standardized coefficients')
plt.title('All Lasso coefficients as a function of the regularization');

In [1]:
3

3